In [1]:
import re
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import pymongo
import json

from nltk.corpus import stopwords
from spacy.en import English
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from spacy.en import STOP_WORDS
nlp = English()
stop = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [2]:
client = pymongo.MongoClient('54.201.199.246', 27016)

wiki_db = client.wikipedia

wiki_col = wiki_db.my_collection

In [3]:
client.database_names(), wiki_db.collection_names()

(['admin', 'local', 'my_database', 'test', 'wikipedia'], ['my_collection'])

In [4]:
wiki_col.count()

5654

In [5]:
cursor = wiki_col.find()

In [6]:
wiki_df = pd.DataFrame(list(cursor))

In [8]:
wiki_df['main_cat'].value_counts()

Business software    4117
Machine learning     1537
Name: main_cat, dtype: int64

In [9]:
wiki_df.head()

,_id,article,content,main_cat,page_id,sub_cat
0,5a15de5730b30c01325f0260,Business software,merge enterprise software date october softw...,Business software,1037763,Business software
1,5a15de5830b30c01325f0261,AccuSystems,multiple issue orphan date february notabili...,Business software,41270069,Business software
2,5a15de5830b30c01325f0262,Active policy management,active policy management business orient ent...,Business software,5211212,Business software
3,5a15de5830b30c01325f0263,Alexandria (library software),use alexandria alexandria browser base softw...,Business software,28502793,Business software
4,5a15de5930b30c01325f0264,Alteryx,infobox company name alteryx logo file alter...,Business software,44133735,Business software


In [10]:
wiki_df.drop_duplicates(subset=['page_id'], inplace=True)

## Use TIFIDF to vectorize words

In [12]:
tfidf_vectorizer = TfidfVectorizer(min_df = 20, stop_words = 'english')

article_term_matrix_sps = tfidf_vectorizer.fit_transform(wiki_df.content)

article_term_matrix_df = pd.DataFrame(article_term_matrix_sps.toarray(),
                                       index=wiki_df.index,
                                       columns=tfidf_vectorizer.get_feature_names())

In [ ]:
article_term_matrix_df.head()


In [ ]:
# pd.concat([wiki_df.article, wiki_df.content, article_term_matrix_df], axis=1).sample(4)

## Use SVD to reduce number of features

In [13]:
from sklearn.decomposition import TruncatedSVD

In [14]:
n_components = 500
SVD = TruncatedSVD(n_components)
component_names = ["component_"+str(i+1) for i in range(n_components)]

In [16]:
svd_matrix = SVD.fit_transform(article_term_matrix_df)

In [ ]:
sum(SVD.explained_variance_ratio_)

In [17]:
svd_df = pd.DataFrame(svd_matrix,
                      index=article_term_matrix_df.index,
                      columns=component_names)
svd_df['article'] = wiki_df.article

vocabulary_expression = pd.DataFrame(SVD.components_,
                                     index=component_names,
                                     columns=tfidf_vectorizer.get_feature_names()).T

In [18]:
svd_df.head()

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_492,component_493,component_494,component_495,component_496,component_497,component_498,component_499,component_500,article
0,0.404951,-0.051309,-0.040997,0.105726,0.005325,0.123327,0.257478,-0.055066,-0.030250,0.069374,...,-0.010739,-0.010833,-0.023893,-0.023662,0.003200,-0.016697,0.034757,0.009308,0.019910,Business software
1,0.392458,-0.058191,-0.175200,0.280272,-0.069831,-0.002597,-0.034980,-0.052762,-0.030091,-0.091638,...,-0.014407,-0.008930,0.022893,-0.022598,0.018009,0.003248,0.013680,0.011187,-0.004865,AccuSystems
2,0.182103,-0.024958,-0.010863,0.079786,0.076378,0.052743,0.067091,0.026575,-0.068131,0.055673,...,-0.016108,-0.009719,0.012636,-0.013827,0.012805,0.001512,0.005479,-0.000555,0.008461,Active policy management
3,0.222011,-0.035081,-0.032633,-0.016740,0.000543,0.002462,-0.014987,-0.016434,-0.001359,0.003528,...,-0.006281,0.015953,-0.012347,0.000610,0.000705,-0.024892,0.001886,-0.020234,0.024531,Alexandria (library software)
4,0.328022,-0.045848,-0.136050,0.251924,-0.101908,-0.008932,-0.081420,-0.045797,-0.059878,-0.074248,...,-0.011901,0.003449,-0.022862,-0.007964,0.026287,0.029398,-0.009497,-0.012136,-0.020458,Alteryx


In [ ]:
for i in range(1,11):
    vocabulary_expression['abs_component_{}'.format(i)] = \
    np.abs(vocabulary_expression['component_{}'.format(i)])

In [ ]:
vocabulary_expression['abs_component_1'].sort_values(ascending=False).head(7)

## Create function to search for top 5 related articles

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
def search_for_pages(search_terms):
    '''
    Takes search terms and returns the top 5 articles within the wikipedia corpus 
    that relate to that search terms based on cosine similarity.
    
    Params
    ------
    search_terms: str
    A string of words  
    
    Returns
    -------
    A Dataframe of the top 5 articles with the highest cosine similarities.
     
    '''
    
    temp_svd_df = svd_df.copy()
    
    search_terms = [search_terms]

    search_terms_encoded = tfidf_vectorizer.transform(search_terms)
    
    search_term_svd_vector = SVD.transform(search_terms_encoded)
    
    temp_svd_df['cosine_sim'] = cosine_similarity(temp_svd_df.drop('article', axis=1), search_term_svd_vector)
    
    return temp_svd_df[['article', 'cosine_sim']].sort_values('cosine_sim', ascending=False).head(5)

In [21]:
search_for_pages('There are two types of investor apps: Native investor apps and HTML5 investor apps. Most investor apps offer access to public company content such as stock quotes, corporate materials')

,article,cosine_sim
2879,Investor application,0.833884
1625,Alpha capture system,0.393927
109,Fundamental Analysis Software,0.382128
181,PandaDoc,0.371676
1643,FatKat (investment software),0.345342


In [24]:
search_for_pages('Artificial intelligence')

,article,cosine_sim
4882,AAAI Conference on Artificial Intelligence,0.821924
4884,Conference on Artificial General Intelligence,0.697031
4887,Dartmouth workshop,0.665428
4891,International Joint Conference on Artificial I...,0.624944
4334,Language Acquisition Device (computer),0.613826
